# Fun with language modelling

* [Unreasonable effectiveness of RNN](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) (Andrej Karpathy)
* [Официальный гайд от TensorFlow](https://www.tensorflow.org/tutorials/sequences/recurrent)

---

## Препроцессинг (2 балл)

Возьмите какие-нибудь сырые данные. Википедия, «Гарри Поттер», «Игра Престолов», твиты Тинькова — что угодно.

Давайте для простоты делать char-level модель. Сопоставьте всем различным символам свой номер. Удобно это хранить просто в питоновском словаре (`char2idx`). Для генерации вам потребуется ещё и обратный словарь (`idx2char`).

Клёво будет ещё написать класс, который делает токенизацию и детокенизацию.

In [14]:
import tensorflow as tf
import numpy as np

In [15]:
filename = 'HP1RUS.TXT'
with open(filename, 'r', encoding = "utf-8") as file:
    text = file.read()
    file.close()

In [16]:
vocab = set(text)
vocab_size = len(vocab)

char2idx = {char:idx for idx, char in enumerate(vocab)}
idx2char = {idx:char for idx, char in enumerate(vocab)}

text_as_int = np.array([char2idx[c] for c in text])

In [25]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(data)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

TypeError: batch() got an unexpected keyword argument 'drop_remainder'

In [26]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

NameError: name 'sequences' is not defined

## Модель (1 балл)

Примерно такое должно зайти:

* Эмбеддинг
* LSTM / GRU
* Дропаут
* Линейный слой
* Softmax

In [35]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np

/home/nullkatar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [36]:
embedding_matrix = np.identity(vocab_size)

hidden_size = vocab_size / 2 
seq_length = 1000

dataset = tf.data.Dataset.from_tensor_slices((data[:-1], data[1:])).shuffle(BATCH_SIZE).repeat().batch(BATCH_SIZE)

IndexError: list index out of range

In [38]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = chunks.map(split_input_target)

NameError: name 'chunks' is not defined

In [37]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = map(split_input_target, data)
dataset = data.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

NameError: name 'split_input_target' is not defined

In [ ]:
iterator = dataset.make_one_shot_iterator()
get_next = iterator.get_next()

inputs, labels = get_next

embedding_mtx = tf.get_variable(name="embedding_mtx",
                            shape=embedding_matrix.shape,
                            initializer=tf.constant_initializer(embedding_matrix),
                            trainable=False)

inputs_embedded = tf.nn.embedding_lookup(params=embedding_mtx, ids=inputs)

In [ ]:
outputs, states = tf.nn.dynamic_rnn(cell=tf.nn.rnn_cell.LSTMCell(hidden_size), inputs=inputs, dtype=tf.int32)
outputs = tf.layers.flatten(outputs)
logits = tf.layers.dense(outputs, units=vocab_size)

probs = tf.nn.sigmoid(logits)

## Обучение (3 балла)

* Делайте сэмплирование предложений фиксированной длины из вашего корпуса. Можете как нарезать их изначально, так и написать генератор.
* Используйте teacher forcing.
* Выход модели — это one-hot вход, смещенный на одну позиию.
* Функция потерь: кроссэнтропия.
* Не забудьте мониторить и валидацию, и train.

In [ ]:
batch_size = # ...
sequence_len = # ...
learning_rate = # ...

# ...

## Спеллчекер (1 балла)

Из языковой модели можно сделать простенький спеллчекер: можно визуализировать лоссы на каждом символе.

Бонус: можете усреднить перплексии по словам и выделять их, а не отдельные символы.

In [55]:
from IPython.core.display import display, HTML

def print_colored(sequence, intensities, delimeter=''):
    html = delimeter.join([
        f'<span style="background: rgb({255}, {255-x}, {255-x})">{c}</span>'
        for c, x in zip(sequence, intensities) 
    ])
    display(HTML(html))

print_colored('Налейте мне экспрессо'.split(), [0, 0, 100], ' ')

sequence = 'Эту домашку нужно сдать втечении двух недель'
intensities = [0]*len(sequence)
intensities[25] = 50
intensities[26] = 60
intensities[27] = 70
intensities[31] = 150
print_colored(sequence, intensities)

In [ ]:
# ...

## Генерация предложений (3 балла)

* Поддерживайте hidden state при генерации. Не пересчитывайте ничего больше одного раза.
* Прикрутите температуру: это когда при сэмплировании все логиты (то, что перед софтмаксом) делятся на какое-то число (по умолчанию 1, тогда ничего не меняется). Температура позволяет делать trade-off между разнообразием и правдоподобием (подробнее — см. блог Карпатого).
* Ваша реализация должна уметь принимать строку seed — то, с чего должно начинаться сгенерированная строка.

Если сделаете все вышеперечисленное, то получите 2 балла. Если сделаете хоть какую-то генерацию, то 1 балл.

In [ ]:
def sample(length, temperature=1, seed=''):
    # ...